In [1]:
# install missing packages
%pip install plotly
%pip install py-cpuinfo
%pip install pandas-datareader

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# check system details
import os
import psutil
import cpuinfo

ram_info = psutil.virtual_memory()
print(f"Total RAM: {ram_info.total / 1024 / 1024 / 1024:.2f} GB")
print(f"Available RAM: {ram_info.available / 1024 / 1024 / 1024:.2f} GB")
print(f"Used RAM: {ram_info.used / 1024 / 1024 / 1024:.2f} GB")
print(f"Percentage Usage Of RAM: {ram_info.percent}%")
print(f"CPU Cores: {os.cpu_count()}")
print(f"CPU Speed: {cpuinfo.get_cpu_info()['hz_actual_friendly']}")
disk_info = psutil.disk_usage(os.getcwd())
print(f"Total Disk: {disk_info.total / 1024 / 1024 / 1024:.2f} GB")
print(f"Available Disk: {disk_info.free / 1024 / 1024 / 1024:.2f} GB")
print(f"Used Disk: {disk_info.used / 1024 / 1024 / 1024:.2f} GB")
print(f"Percentage Usage Of Disk: {disk_info.percent}%")

Total RAM: 15.47 GB
Available RAM: 9.90 GB
Used RAM: 5.30 GB
Percentage Usage Of RAM: 36.0%
CPU Cores: 4
CPU Speed: 2.5000 GHz
Total Disk: 24.99 GB
Available Disk: 17.04 GB
Used Disk: 7.95 GB
Percentage Usage Of Disk: 31.8%


In [3]:
# import requirements
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score
from gbm_classifier import Classification

In [4]:
# get the data
crime = pd.read_csv("survival.csv")

In [5]:
# split up the data into training and testing
y = crime[["Survived"]]
X = crime.drop(columns="Survived")
trainX = X.head(int(0.8 * X.shape[0]))
trainy = y.head(int(0.8 * y.shape[0]))
testX = X.tail(int(0.2 * X.shape[0])).reset_index(drop=True)
testy = y.tail(int(0.2 * y.shape[0])).reset_index(drop=True)

In [6]:
# build the model
print("\n---- Titanic Classification Analysis ----\n")
model = Classification(
    name="XGBoost Without Feature Engineering", 
    path=None,
    rename=False, 
    time=False, 
    binary=True, 
    imputation=True, 
    variance=True,
    atwood=False,
    binning=False,
    reciprocal=False, 
    interaction=False, 
    selection=False,
    plots=True,
)
try:
    model.load()  # load the machine learning pipeline
    predictions = model.predict(testX)
except:
    model.fit(trainX, trainy)  # build the machine learning pipeline
    predictions = model.predict(testX)
    # print(f"Accuracy: {model.accuracy}")
    # print(f"In Control: {model.in_control}")
    # print("Confusion Matrix:")
    # print(model.confusion)


---- Titanic Classification Analysis ----

Model Training:
> Transforming The Data
> Transforming Categorical Features
> Filling In Missing Values
> Removing Constant Features
> Training XGBoost
1.81 Seconds
Model Indicators:
> Extracting Important Features
0.58 Seconds
Model Prediction:
> Transforming The New Data
> Getting Predictions
0.06 Seconds
Model Monitoring:
> Computing Feature Drift
0.28 Seconds


In [7]:
# model diagnostics
print("Model Indicators:")
for i, indicator in enumerate(model.indicators["Indicator"][:10]):
    print(f"{i+1}. {indicator}")
print(" ")
print("Feature Drift:")
for i, feature in enumerate(model.drift.loc[model.drift["pvalue"] < 0.05, "Feature"][:10]):
    print(f"{i+1}. {feature}")
if model.drift.loc[model.drift["pvalue"] < 0.05].shape[0] == 0:
    print("None")

Model Indicators:
1. Sex_male
2. Title_Rev.
3. Title_Mr.
4. Sex_female
5. Pclass_3
6. Pclass_2
7. Title_Master.
8. Ticket_4
9. Cabin_N
10. Ticket_1
 
Feature Drift:
None


In [8]:
# score the model
accuracy = accuracy_score(
    y_true=testy.iloc[:,0].to_numpy(),
    y_pred=predictions,
)

print(f"Accuracy: {accuracy}")

Accuracy: 0.8258426966292135


In [9]:
# show the confusion matrix
ytest = testy.iloc[:,0].to_numpy()
labels = np.unique(np.concatenate((predictions, ytest)))
confusion = confusion_matrix(
    y_true=ytest,   # rows
    y_pred=predictions,  # columns
    labels=labels,
)
confusion = pd.DataFrame(
    confusion, 
    columns=labels, 
    index=labels,
)
print("Confusion Matrix:")
pd.set_option("display.width", 1000)
print(confusion)

Confusion Matrix:
    0   1
0  87  10
1  21  60


In [10]:
# save the machine learning pipeline
model.dump()

In [11]:
# refit the model to include the test data
model.refit(testX, testy)

Model Retraining:
> Transforming The Updated Data
> Transforming Categorical Features
> Filling In Missing Values
> Removing Constant Features
> Training XGBoost
1.21 Seconds
Model Indicators:
> Extracting Important Features
0.2 Seconds
